# Plot 6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("mime")
import h5py
from scipy.optimize import curve_fit
import lmfit
import os

colors = ['#bb0000', '#00bb00', "#0000bb", '#bbbb00', '#bb00bb', "#00bbbb", '#bbbbbb', '#770000', '#007700', "#000077", '#777700', '#770077', "#007777", '#777777', '#440000', '#004400', "#000044", '#444400', '#440044', "#0044444", '#444444','#000000']

In [ ]:
baseSize = (8, 6)  # Base size of a subplot

def sbPlot(n):
    if (n == 1): return 1, 1
    if (n <= 2): return 1, 2
    if (n <= 4): return 2, 2
    if (n <= 6): return 3, 2
    if (n <= 9): return 3, 3
    if (n <= 12): return 4, 3
    return 0, 0

def sbPlotSize(n):
    x, y = sbPlot(n)
    return baseSize[0] * y, baseSize[1] * x

In [ ]:
h5f = h5py.File('GleebleData.h5','r')
allData = h5f['all'][:]
shortData = h5f['short'][:]
h5f.close()

In [ ]:
allData.shape, shortData.shape

Remove first point of each curve, where $\varepsilon^p=0$

In [ ]:
allData = allData[allData[:,0] != 0]
shortData = shortData[shortData[:,0]!=0]

In [ ]:
strains = np.unique(shortData[:,0])
allStrains = np.unique(shortData[:,0])
epsps = np.unique(shortData[:,1])
temperatures = np.unique(shortData[:,2])
nEps = len(strains)
nEpsp = len(epsps)
nTemp = len(temperatures)
#strains, epsps, temperatures, nEps, nEpsp, nTemp

In [ ]:
loadModels = ['PTM-6', 'JC-6', 'MZA-6', 'HS-6', 'AR-6', '3-15-7-1-sigmoid']
dataPaths = ['PTM', 'JC', 'MZA', 'HS', 'AR', 'ANN/ANN-6']
mod = 5
model = loadModels[mod]
dataPath = dataPaths[mod]
print("Model : %s/%s"%(dataPath,model))

In [ ]:
h5f = h5py.File(dataPath+'/'+model+'-6.h5','r')
data = h5f['data'][:]
h5f.close()
data.shape

In [ ]:
# Plot the curves
from matplotlib.lines import Line2D
def create_dummy_line(**kwds):
    return Line2D([], [], **kwds)

plt.figure(figsize = sbPlotSize(nEpsp))
plt.rc('text', usetex = True)
idx = 1
plt.subplots_adjust(hspace = 0.3)
for epsp in list(epsps):
    xs, ys = sbPlot(nEpsp)
    plt.subplot(xs, ys, idx)
    sbdata = shortData[shortData[:,1]==epsp]
    sndata = data[data[:,1]==epsp]
    cl =0
    for temp in list(temperatures):
        sbdata1 = sbdata[sbdata[:,2]==temp]
        sndata1 = sndata[sndata[:,2]==temp]
        plt.plot(sbdata1[:,0], sbdata1[:,3], colors[cl], marker = 's', markersize = 5, linestyle = 'none')
        plt.plot(sndata1[:,0], sndata1[:,3], colors[cl], linewidth = 2.5)
        plt.rcParams['xtick.labelsize'] = 16
        plt.rcParams['ytick.labelsize'] = 16
        cl +=1
    plt.xlim(0, 0.7)
    plt.ylim(bottom=0)
    plt.xlabel(r'strain $\varepsilon$', fontsize = 16) # Labels the x axis
    plt.ylabel(r'flow stress $\sigma^y$ (MPa)', fontsize = 16) # Labels the y axis
    plt.title(r'strain rate $\dot{\varepsilon} = ' + str(epsp) + '$ s$^{-1}$', fontsize = 16) # Self explicit command
    idx += 1
    
legendLines = []
cl = 0
for temp in list(temperatures):
    legendLines.append((r'$T=$' + str(int(temp)) + r'$^{\circ}$C', {'color':colors[cl], 'linestyle':'-', 'linewidth':2.5, 'marker':'s'}))
    cl += 1

plt.legend([create_dummy_line(**l[1]) for l in legendLines],[l[0] for l in legendLines], 
           loc = 'upper center', fontsize = 12, ncols = 5, bbox_to_anchor = (0.0, -0.2), shadow = False)

plt.savefig("CompExp-"+model+".svg")
plt.show()

In [ ]:
ref = allData[allData[:,0] != 0]
EAAR = np.sum(np.abs((ref[:,3] - data[:,3])/(ref[:,3])))*100/ref.shape[0]
RMSE = np.sqrt(np.sum((ref[:,3] - data[:,3])**2)/ref.shape[0])
print('RMSE = %g' %(RMSE)+' MPa')
print("EAAR = %g" %(EAAR) + ' %')

In [ ]:
# Plot the curves
stp = 20
m = np.min([allData[::stp,3].min(), data[::stp,3].min()])
M = np.max([allData[::stp,3].max(), data[::stp,3].max()])
plt.figure(figsize = (11.69, 8.27))
plt.rc('text', usetex = True)
plt.plot(allData[::stp,3], data[::stp,3], label= r'$Data\ points$', marker='o', linestyle = 'none', color='blue')
plt.plot([m,M],[m,M], linewidth = 2, color='red')
#plt.legend(loc = 'upper left',fancybox = True, numpoints = 1, fontsize = 10)
plt.xlabel(r'Experimental flow stress $\sigma^e$', fontsize = 16) # Labels the x axis
plt.ylabel(r'Predicted flow stress $\sigma^y$', fontsize = 16) # Labels the y axis
plt.title(r'Correlation of the '+model+' model', fontsize = 16) # Self explicit command
plt.savefig("CorrExp-"+model+".svg")
plt.show()